# Venture Funding with Deep Learning



In [1]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import keras_tuner as kt #pip install -q -U keras-tuner
import keras

2022-10-21 12:55:40.941258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [2]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv('./Resources/applicants_data.csv')

# Review the DataFrame
applicant_data_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [4]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN','NAME'])

# Review the DataFrame
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [5]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=='object'].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [7]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [8]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df.head()


/Users/katerina/.pyenv/versions/3.10.5/envs/myenv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [9]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([encoded_df,applicant_data_df.drop(columns=categorical_variables)],axis=1)

# Review the Dataframe
encoded_df.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [10]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [11]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns='IS_SUCCESSFUL')

# Review the features DataFrame
X.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590


### Step 6: Split the features and target sets into training and testing datasets.


In [12]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [14]:
# Define the the number of inputs (features) to the model
number_input_features = len(X.columns)

# Review the number of features
number_input_features


116

In [15]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [16]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features+number_output_neurons)//2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


58

In [17]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1+number_output_neurons)//2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


29

In [18]:
# Create the Sequential model instance
nn = Sequential()


2022-10-21 12:55:50.827245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))


In [20]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation='relu'))


In [21]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation='sigmoid'))


In [22]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [23]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [24]:
# Fit the model using 50 epochs and the training data
fit_model=nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5800 - accuracy: 0.7164
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7302
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7315
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7320
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5457 - accuracy: 0.7329
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5445 - accuracy: 0.7338
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5437 - accuracy: 0.7329
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5426 - accuracy: 0.7352
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5427 - accuracy: 0.7347
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7361

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [25]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5558 - accuracy: 0.7317 - 405ms/epoch - 2ms/step
Loss: 0.5557965040206909, Accuracy: 0.7316617965698242


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [26]:
import tensorflow as tf
# Set the model's file path
file_path = Path("./Resources/applicants_model.h5")

# Export your model to a HDF5 file
nn.save(file_path)

---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


##### Select the best possible parameters for the model with Keras-Tuner


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
def model_builder(hp):
  model = keras.Sequential()

  model.add(keras.layers.Dense(
    units=(number_input_features+1)//2, 
    activation='relu', 
    input_dim=number_input_features))
  # Tune the number of units in the first Dense layer
  # Choose an optimal value
  #hp_units = hp.Int('units', min_value=10, max_value=((number_input_features+1)//2+1)//2)
  model.add(
            keras.layers.Dense(
                hp.Int('units', min_value=(((number_input_features+1)//2)+1)//2, max_value=((number_input_features+1)//2)),
                activation='relu') #hp.Choice(
                   # 'dense_activation',
                   # values=['relu']),
           )
        
  #model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(
    units=1, 
    activation='sigmoid')) #hp.Choice(
                    #'dense_activation',
                    #values=['sigmoid'])))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.summary()
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [30]:
tuner = kt.Hyperband(model_builder,
                     objective='accuracy',
                     max_epochs=10,
                     factor=3,
                     overwrite=True)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 58)                6786      
                                                                 
 dense_4 (Dense)             (None, 29)                1711      
                                                                 
 dense_5 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [31]:
tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 30 Complete [00h 00m 17s]
accuracy: 0.7311336994171143

Best accuracy So Far: 0.7324457168579102
Total elapsed time: 00h 04m 25s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 30 and the optimal learning rate for the optimizer
is 0.001.



In [32]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 58)                6786      
                                                                 
 dense_4 (Dense)             (None, 30)                1770      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,587
Trainable params: 8,587
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
644/644 [==============================] - 2s 2ms/step - loss: 0.5798 - accuracy: 0.7186 - val_loss: 0.5654 - val_accuracy: 0.7341
Epoch 2/50
644/644 [==============================] - 1s 2ms/step - loss: 0.5565 - accuracy: 0.7276 - val_loss: 0.5555 - val_accuracy: 0.7349
Epoch 3/50
644/644 [============

In [33]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train_scaled, y_train, epochs=best_epoch, validation_split=0.2)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 58)                6786      
                                                                 
 dense_7 (Dense)             (None, 30)                1770      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,587
Trainable params: 8,587
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
644/644 [==============================] - 2s 2ms/step - loss: 0.5795 - accuracy: 0.7117 - val_loss: 0.5577 - val_accuracy: 0.7331
Epoch 2/10
644/644 [==============================] - 1s 2ms/step - loss: 0.5561 - accuracy: 0.7258 - val_loss: 0.5576 - val_accuracy: 0.7312
Epoch 3/10
644/644 [============

In [34]:
eval_result = hypermodel.evaluate(X_test_scaled, y_test)
print("[test loss, test accuracy]:", eval_result)


268/268 [==============================] - 0s 1ms/step - loss: 0.5532 - accuracy: 0.7301
[test loss, test accuracy]: [0.5532194972038269, 0.7301457524299622]


Based on the outcome from the Keras tuner, the next alternative models will use 30 as the number of nodes in the first hidden layer and the learning rate for the optimizer 0.001.

### Alternative Model 1

Based on the previous results I am gonna select the parameters:

for the first alternative model

and 

for the second alternative model.

In [35]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [36]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [59]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = 30

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

30

In [60]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = (hidden_nodes_layer1_A1+number_output_neurons_A1)//2

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A1

15

In [61]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A1 = (hidden_nodes_layer2_A1+number_output_neurons_A1)//2

# Review the number of hidden nodes in the third layer
hidden_nodes_layer3_A1

8

In [62]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [63]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1,input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation='relu'))

# Third hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer3_A1, activation='relu'))

# Output layer
nn_A1.add(Dense(units=number_output_neurons_A1, activation='sigmoid'))


# Check the structure of the model
nn_A1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 30)                3510      
                                                                 
 dense_19 (Dense)            (None, 15)                465       
                                                                 
 dense_20 (Dense)            (None, 8)                 128       
                                                                 
 dense_21 (Dense)            (None, 1)                 9         
                                                                 
Total params: 4,112
Trainable params: 4,112
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Compile the Sequential model
nn_A1.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])


In [67]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled,y_train,epochs=10)


Epoch 1/10
804/804 [==============================] - 3s 2ms/step - loss: 0.5641 - accuracy: 0.7252
Epoch 2/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7288
Epoch 3/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7299
Epoch 4/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7320
Epoch 5/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7324
Epoch 6/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7336
Epoch 7/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7343
Epoch 8/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5431 - accuracy: 0.7354
Epoch 9/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7347
Epoch 10/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7347

#### Alternative Model 2

In [68]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [69]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [70]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = 30

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

30

In [71]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = (hidden_nodes_layer1_A2+number_output_neurons_A2)//2

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A2

15

In [72]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A2 = (hidden_nodes_layer2_A2+number_output_neurons_A2)//2

# Review the number of hidden nodes in the third layer
hidden_nodes_layer3_A2

8

In [73]:
# Define the number of hidden nodes for the fourth hidden layer
hidden_nodes_layer4_A2 = (hidden_nodes_layer3_A2+number_output_neurons_A2)//2

# Review the number of hidden nodes in the fourth layer
hidden_nodes_layer4_A2

4

In [74]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [75]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2,input_dim=number_input_features,activation='relu'))
# Second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2,activation='relu'))
# Third hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2,activation='relu'))

# Fourth hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2,activation='relu'))

# Output layer
nn_A2.add(Dense(units=number_output_neurons_A2,activation='sigmoid'))

# Check the structure of the model
nn_A2.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 30)                3510      
                                                                 
 dense_23 (Dense)            (None, 15)                465       
                                                                 
 dense_24 (Dense)            (None, 8)                 128       
                                                                 
 dense_25 (Dense)            (None, 4)                 36        
                                                                 
 dense_26 (Dense)            (None, 1)                 5         
                                                                 
Total params: 4,144
Trainable params: 4,144
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Compile the model
from tensorflow import keras
nn_A2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])


In [77]:
# Fit the model
fit_model_A2=nn_A2.fit(X_train_scaled,y_train,epochs=50)


Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5865 - accuracy: 0.7088
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7273
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7301
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7324
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5456 - accuracy: 0.7320
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5441 - accuracy: 0.7331
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7348
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7345
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5425 - accuracy: 0.7345

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [78]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5558 - accuracy: 0.7317 - 341ms/epoch - 1ms/step
Loss: 0.5557965040206909, Accuracy: 0.7316617965698242


In [79]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 1s - loss: 0.5548 - accuracy: 0.7259 - 539ms/epoch - 2ms/step
Loss: 0.5548203587532043, Accuracy: 0.7259474992752075


In [80]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.5520 - accuracy: 0.7306 - 428ms/epoch - 2ms/step
Loss: 0.5519663691520691, Accuracy: 0.7306122183799744


### Step 3: Save each of your alternative models as an HDF5 file.


In [81]:
# Set the file path for the first alternative model
file_path = Path('./Resources/applicants_model_A1.h5')

# Export your model to a HDF5 file
nn_A1.save(file_path)


In [82]:
# Set the file path for the second alternative model
file_path = Path('./Resources/applicants_model_A2.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path)
